# 2️⃣ Train: 모델 학습 파이프라인

이 노트북은 다음을 수행합니다:
1. 모델 프루닝 (Pruning)
2. 지식 증류 파인튜닝 (Output KD, Feature KD, FaKD) - **200 에폭 × 5회 = 총 1000 에폭**
3. 모델 재구성 (Rebuild)
4. 학습 결과 시각화

---
## ⚙️ 0. 경로 설정 (이 셀만 수정하세요!)

In [ ]:
# ============================================================
# 🔧 사용자 설정 - 1_setup.ipynb와 동일하게!
# ============================================================
BASE_DIR = "my_ssl"
VENV_NAME = "ssl_env"
PYTHON_VERSION = "3.12"
REPO_NAME = "uos-ssl-project"

# ============================================================
# 🔒 아래는 자동 설정 (수정 불필요)
# ============================================================
import os
import sys
import subprocess
os.environ['MPLBACKEND'] = 'Agg'

home_dir = os.path.expanduser('~')
base_path = os.path.join(home_dir, BASE_DIR)
venv_path = os.path.join(base_path, VENV_NAME)
site_packages = os.path.join(venv_path, 'lib', f'python{PYTHON_VERSION}', 'site-packages')
python_exec = os.path.join(venv_path, 'bin', 'python')
project_dir = os.path.join(base_path, REPO_NAME)
ai_dir = os.path.join(project_dir, 'ai')

if site_packages not in sys.path:
    sys.path.insert(0, site_packages)

%cd {ai_dir}

def run_cmd(cmd, task_name):
    """명령어 실행 후 에러 체크"""
    print(f"🚀 {task_name} 시작...")
    print("="*60)
    result = subprocess.run(cmd, shell=True)
    if result.returncode != 0:
        print(f"\n❌ {task_name} 실패! (exit code: {result.returncode})")
        return False
    print(f"\n✅ {task_name} 완료!")
    return True

print("="*60)
print("✅ 환경 설정 완료")
print("="*60)
print(f"AI Dir:  {ai_dir}")
print(f"Python:  {python_exec}")

# 경로 검증
print("\n📁 경로 검증:")
paths_to_check = [
    (ai_dir, "AI 디렉토리"),
    (python_exec, "Python 실행파일"),
    (os.path.join(ai_dir, "weights"), "weights 폴더"),
    (os.path.join(ai_dir, "config_catanet.yml"), "config 파일"),
]
all_paths_ok = True
for path, name in paths_to_check:
    if os.path.exists(path):
        print(f"   ✅ {name}")
    else:
        print(f"   ❌ {name}: {path}")
        all_paths_ok = False

if not all_paths_ok:
    print("\n⚠️ 경로 오류! 1_setup.ipynb를 먼저 실행하세요.")

## 2.1 이전 결과 정리 (선택)

In [ ]:
# ⚠️ 주의: 이전 학습 결과를 삭제합니다!
# 필요한 경우에만 주석 해제하여 실행하세요.

# import shutil
# files_to_remove = [
#     'weights/finetuned_output.pth',
#     'weights/finetuned_feature.pth', 
#     'weights/finetuned_fakd.pth',
# ]
# for f in files_to_remove:
#     if os.path.exists(f):
#         os.remove(f)
#         print(f"삭제: {f}")
# for folder in ['results/logs', 'results/plots/psnr', 'results/plots/comparison']:
#     if os.path.exists(folder):
#         shutil.rmtree(folder)
#         os.makedirs(folder)
#         print(f"초기화: {folder}")

print("💡 이전 결과를 삭제하려면 위 주석을 해제하세요.")

## 2.2 모델 프루닝 (Pruning)

In [ ]:
# ⚠️ 프루닝은 이미 완료되어 있으므로 주석 처리됨

# cmd = f"{python_exec} run_catanet_pruning.py --config config_catanet.yml"
# run_cmd(cmd, "모델 프루닝")

print("💡 프루닝이 필요하면 위 주석을 해제하세요.")
print("\n📁 기존 파일 확인:")
for f in ['weights/catanet_pruned.pth', 'weights/catanet_pruning_masks.pth']:
    if os.path.exists(f):
        size = os.path.getsize(f) / (1024*1024)
        print(f"   ✅ {f} ({size:.1f} MB)")
    else:
        print(f"   ❌ {f} (없음)")

---
## 2.3 지식 증류 - Output KD (200 에폭 × 5회 = 1000 에폭)

In [ ]:
# Output KD - Round 1/5
cmd = f"""{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_output.pth \
    --distillation_type output \
    --epochs 200 --lr 1e-4 --alpha 0.8"""
run_cmd(cmd, "Output KD [1/5]")

In [ ]:
# Output KD - Round 2/5
cmd = f"""{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_output.pth \
    --distillation_type output \
    --epochs 200 --lr 1e-4 --alpha 0.8"""
run_cmd(cmd, "Output KD [2/5]")

In [ ]:
# Output KD - Round 3/5
cmd = f"""{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_output.pth \
    --distillation_type output \
    --epochs 200 --lr 1e-4 --alpha 0.8"""
run_cmd(cmd, "Output KD [3/5]")

In [ ]:
# Output KD - Round 4/5
cmd = f"""{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_output.pth \
    --distillation_type output \
    --epochs 200 --lr 1e-4 --alpha 0.8"""
run_cmd(cmd, "Output KD [4/5]")

In [ ]:
# Output KD - Round 5/5
cmd = f"""{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_output.pth \
    --distillation_type output \
    --epochs 200 --lr 1e-4 --alpha 0.8"""
run_cmd(cmd, "Output KD [5/5]")

In [ ]:
# Output KD 그래프
import pandas as pd
import matplotlib.pyplot as plt

log_path = 'results/logs/output_kd.csv'
if os.path.exists(log_path):
    df = pd.read_csv(log_path)
    print(f"📊 Output KD: {len(df)} 에폭")
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    axes[0].plot(df['epoch'], df['total_loss'], label='Total', color='blue')
    axes[0].set_xlabel('Epoch'); axes[0].set_ylabel('Loss')
    axes[0].set_title('Output KD - Loss'); axes[0].legend(); axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(df['epoch'], df['val_psnr'], color='purple', linewidth=2)
    axes[1].axhline(y=df['val_psnr'].max(), color='red', linestyle='--', alpha=0.5)
    axes[1].set_xlabel('Epoch'); axes[1].set_ylabel('PSNR (dB)')
    axes[1].set_title(f'Output KD - PSNR (Best: {df["val_psnr"].max():.2f})'); axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    os.makedirs('results/plots/psnr', exist_ok=True)
    plt.savefig('results/plots/psnr/output_kd.png', dpi=150)
    plt.show()
else:
    print("❌ 로그 없음")

---
## 2.4 지식 증류 - Feature KD (200 에폭 × 5회 = 1000 에폭)

In [ ]:
# Feature KD - Round 1/5
cmd = f"""{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_feature.pth \
    --distillation_type feature \
    --epochs 200 --lr 1e-4 --alpha 0.5 --beta 1"""
run_cmd(cmd, "Feature KD [1/5]")

In [ ]:
# Feature KD - Round 2/5
cmd = f"""{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_feature.pth \
    --distillation_type feature \
    --epochs 200 --lr 1e-4 --alpha 0.5 --beta 1"""
run_cmd(cmd, "Feature KD [2/5]")

In [ ]:
# Feature KD - Round 3/5
cmd = f"""{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_feature.pth \
    --distillation_type feature \
    --epochs 200 --lr 1e-4 --alpha 0.5 --beta 1"""
run_cmd(cmd, "Feature KD [3/5]")

In [ ]:
# Feature KD - Round 4/5
cmd = f"""{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_feature.pth \
    --distillation_type feature \
    --epochs 200 --lr 1e-4 --alpha 0.5 --beta 1"""
run_cmd(cmd, "Feature KD [4/5]")

In [ ]:
# Feature KD - Round 5/5
cmd = f"""{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_feature.pth \
    --distillation_type feature \
    --epochs 200 --lr 1e-4 --alpha 0.5 --beta 1"""
run_cmd(cmd, "Feature KD [5/5]")

In [ ]:
# Feature KD 그래프
log_path = 'results/logs/feature_kd.csv'
if os.path.exists(log_path):
    df = pd.read_csv(log_path)
    print(f"📊 Feature KD: {len(df)} 에폭")
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    axes[0].plot(df['epoch'], df['total_loss'], label='Total', color='blue')
    axes[0].set_xlabel('Epoch'); axes[0].set_ylabel('Loss')
    axes[0].set_title('Feature KD - Loss'); axes[0].legend(); axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(df['epoch'], df['val_psnr'], color='purple', linewidth=2)
    axes[1].axhline(y=df['val_psnr'].max(), color='red', linestyle='--', alpha=0.5)
    axes[1].set_xlabel('Epoch'); axes[1].set_ylabel('PSNR (dB)')
    axes[1].set_title(f'Feature KD - PSNR (Best: {df["val_psnr"].max():.2f})'); axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('results/plots/psnr/feature_kd.png', dpi=150)
    plt.show()
else:
    print("❌ 로그 없음")

---
## 2.5 지식 증류 - FaKD (200 에폭 × 5회 = 1000 에폭)

In [ ]:
# FaKD - Round 1/5
cmd = f"""{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_fakd.pth \
    --distillation_type fakd \
    --epochs 200 --lr 1e-4 --alpha 0.5 --beta 100"""
run_cmd(cmd, "FaKD [1/5]")

In [ ]:
# FaKD - Round 2/5
cmd = f"""{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_fakd.pth \
    --distillation_type fakd \
    --epochs 200 --lr 1e-4 --alpha 0.5 --beta 100"""
run_cmd(cmd, "FaKD [2/5]")

In [ ]:
# FaKD - Round 3/5
cmd = f"""{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_fakd.pth \
    --distillation_type fakd \
    --epochs 200 --lr 1e-4 --alpha 0.5 --beta 100"""
run_cmd(cmd, "FaKD [3/5]")

In [ ]:
# FaKD - Round 4/5
cmd = f"""{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_fakd.pth \
    --distillation_type fakd \
    --epochs 200 --lr 1e-4 --alpha 0.5 --beta 100"""
run_cmd(cmd, "FaKD [4/5]")

In [ ]:
# FaKD - Round 5/5
cmd = f"""{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_fakd.pth \
    --distillation_type fakd \
    --epochs 200 --lr 1e-4 --alpha 0.5 --beta 100"""
run_cmd(cmd, "FaKD [5/5]")

In [ ]:
# FaKD 그래프
log_path = 'results/logs/fakd_kd.csv'
if os.path.exists(log_path):
    df = pd.read_csv(log_path)
    print(f"📊 FaKD: {len(df)} 에폭")
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    axes[0].plot(df['epoch'], df['total_loss'], label='Total', color='blue')
    axes[0].set_xlabel('Epoch'); axes[0].set_ylabel('Loss')
    axes[0].set_title('FaKD - Loss'); axes[0].legend(); axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(df['epoch'], df['val_psnr'], color='purple', linewidth=2)
    axes[1].axhline(y=df['val_psnr'].max(), color='red', linestyle='--', alpha=0.5)
    axes[1].set_xlabel('Epoch'); axes[1].set_ylabel('PSNR (dB)')
    axes[1].set_title(f'FaKD - PSNR (Best: {df["val_psnr"].max():.2f})'); axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('results/plots/psnr/fakd_kd.png', dpi=150)
    plt.show()
else:
    print("❌ 로그 없음")

---
## 2.6 모델 재구성 (Rebuild)

In [ ]:
print("🔧 모델 재구성")
print("="*60)

rebuild_configs = [
    ('finetuned_output.pth', 'rebuilt_output.pth'),
    ('finetuned_feature.pth', 'rebuilt_feature.pth'),
    ('finetuned_fakd.pth', 'rebuilt_fakd.pth'),
]

for source, target in rebuild_configs:
    source_path = f'weights/{source}'
    target_path = f'weights/{target}'
    
    if os.path.exists(source_path):
        cmd = f"""{python_exec} analysis/rebuild_pruned_model.py \
            --config config_catanet.yml \
            --masks weights/catanet_pruning_masks.pth \
            --source_weights {source_path} \
            --save_path {target_path}"""
        run_cmd(cmd, f"Rebuild {source}")
    else:
        print(f"⚠️ {source} 없음, 건너뜀")

---
## 2.7 최종 비교 그래프

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

print("📊 3가지 KD 비교")
print("="*60)

kd_types = ['output', 'feature', 'fakd']
colors = {'output': 'blue', 'feature': 'green', 'fakd': 'red'}
labels = {'output': 'Output KD', 'feature': 'Feature KD', 'fakd': 'FaKD'}

all_logs = {}
for kd_type in kd_types:
    log_path = f'results/logs/{kd_type}_kd.csv'
    if os.path.exists(log_path):
        df = pd.read_csv(log_path)
        all_logs[kd_type] = df
        print(f"  ✅ {labels[kd_type]}: {len(df)} 에폭, Best PSNR: {df['val_psnr'].max():.4f}")
    else:
        print(f"  ❌ {labels[kd_type]}: 없음")

if len(all_logs) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    for kd_type, df in all_logs.items():
        axes[0].plot(df['epoch'], df['val_psnr'], label=labels[kd_type], color=colors[kd_type], linewidth=1.5)
    axes[0].set_xlabel('Epoch'); axes[0].set_ylabel('PSNR (dB)')
    axes[0].set_title('KD Comparison - PSNR'); axes[0].legend(); axes[0].grid(True, alpha=0.3)
    
    for kd_type, df in all_logs.items():
        axes[1].plot(df['epoch'], df['total_loss'], label=labels[kd_type], color=colors[kd_type], linewidth=1.5)
    axes[1].set_xlabel('Epoch'); axes[1].set_ylabel('Loss')
    axes[1].set_title('KD Comparison - Loss'); axes[1].legend(); axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    os.makedirs('results/plots/comparison', exist_ok=True)
    plt.savefig('results/plots/comparison/kd_comparison.png', dpi=150)
    plt.show()

---
## 2.8 학습 결과 요약

In [ ]:
print("="*60)
print("📊 학습 결과 요약")
print("="*60)

print("\n📁 가중치 파일:")
weight_files = [
    'weights/finetuned_output.pth',
    'weights/finetuned_feature.pth',
    'weights/finetuned_fakd.pth',
]
for f in weight_files:
    if os.path.exists(f):
        size = os.path.getsize(f) / (1024*1024)
        print(f"   ✅ {f} ({size:.1f} MB)")
    else:
        print(f"   ❌ {f}")

print("\n📁 학습 로그:")
for kd in ['output', 'feature', 'fakd']:
    log_path = f'results/logs/{kd}_kd.csv'
    if os.path.exists(log_path):
        df = pd.read_csv(log_path)
        print(f"   ✅ {kd}_kd.csv: {len(df)} 에폭")
    else:
        print(f"   ❌ {kd}_kd.csv")

print("\n" + "="*60)
print("💡 다음: 3_ptl.ipynb → 4_benchmark.ipynb")
print("="*60)